In [1]:
from __future__ import division, print_function
%load_ext autoreload
%autoreload 2

In [2]:
import os.path

import numpy as np
import scipy.io

from irispy import Polyhedron

from director import viewerclient as vc

from pydrake.parsers import PackageMap
from pydrake import getDrakePath
from pydrake import rbtree
from pydrake.solvers import ik
from pydrake.trajectories import PiecewisePolynomial
from pydrake.lcm import DrakeLcm
from pydrake.systems import DrakeVisualizer

import mpc_tools.mpcqp as mqp
from boxatlas import boxatlas as box
from boxatlas.contactstabilization import BoxAtlasContactStabilization

In [153]:
def posture_constraints(robot, state, **kwargs):
    constraints = []
    frame_ids = [
        robot.findFrame("r_hand_mount").get_frame_index(),
        robot.findFrame("r_foot_sole").get_frame_index(),
        robot.findFrame("l_foot_sole").get_frame_index(),
        robot.findFrame("l_hand_mount").get_frame_index()
    ]
    for (i, frame_id) in enumerate(frame_ids):
        constraints.append(
            ik.WorldPositionConstraint(robot, frame_id, np.array([0., 0, 0]),
                                       np.array([state.qlimb[i][0], np.nan, state.qlimb[i][1]]),
                                       np.array([state.qlimb[i][0], np.nan, state.qlimb[i][1]]),
                                       **kwargs))
    for frame_id in [robot.findFrame("r_foot_sole").get_frame_index(), robot.findFrame("l_foot_sole").get_frame_index()]:
        constraints.append(
            ik.WorldEulerConstraint(robot, frame_id, np.array([0., 0, np.pi/2]), np.array([0., 0, np.pi/2]), **kwargs))
    constraints.append(
        ik.WorldCoMConstraint(robot,
                              np.array([state.qcom[0], 0, state.qcom[1]]),
                              np.array([state.qcom[0], 0, state.qcom[1]]),
                              **kwargs))
    return constraints

In [154]:
pm = PackageMap()
model = os.path.join(getDrakePath(), "examples", "Atlas", "urdf", "atlas_minimal_contact.urdf")
pm.PopulateUpstreamToDrake(model)
fp = scipy.io.loadmat(os.path.join(getDrakePath(), "examples", "Atlas", "data", "atlas_fp.mat"))
robot = rbtree.RigidBodyTree(model, package_map=pm, floating_base_type=rbtree.FloatingBaseType.kRollPitchYaw)

In [155]:
treevis = vc.Visualizer()
lcm = DrakeLcm()

In [170]:
xstar = fp["xstar"][:]
xstar[5] = np.pi / 2
xstar[1] = -0.2
qstar = xstar[:robot.number_of_positions()]
vstar = xstar[robot.number_of_positions():]
kincache = robot.doKinematics(qstar)
foot_height = robot.transformPoints(kincache, np.zeros((3, 1)), robot.findFrame("r_foot_sole").get_frame_index(), 0)[2, 0]
xstar[2] -= foot_height

pp = PiecewisePolynomial.ZeroOrderHold([0, 0.001], [xstar, xstar])
drakevis = DrakeVisualizer(robot, lcm)
drakevis.PublishLoadRobot()
drakevis.PlaybackTrajectory(pp)

In [158]:
qstar = xstar[:robot.number_of_positions()]
vstar = xstar[robot.number_of_positions():]
kincache = robot.doKinematics(qstar)

atlas = box.BoxAtlas()
initial_state = box.BoxAtlasState(atlas)
initial_state.qcom = robot.centerOfMass(kincache)[[0, 2]]
initial_state.vcom = np.array([-2, 0])
initial_state.qlimb = [
robot.transformPoints(kincache, np.zeros((3, 1)), robot.findFrame("r_hand_mount").get_frame_index(), 0)[[0, 2], 0],
robot.transformPoints(kincache, np.zeros((3, 1)), robot.findFrame("r_foot_sole").get_frame_index(), 0)[[0, 2], 0],
robot.transformPoints(kincache, np.zeros((3, 1)), robot.findFrame("l_foot_sole").get_frame_index(), 0)[[0, 2], 0],
robot.transformPoints(kincache, np.zeros((3, 1)), robot.findFrame("l_hand_mount").get_frame_index(), 0)[[0, 2], 0]
]

In [164]:
dist_to_wall = 0.7
surfaces = [
    box.Surface(Polyhedron.fromBounds([dist_to_wall, 0], [dist_to_wall, 2]),
            Polyhedron(np.array([[1, -1], [1, 1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-2 * dist_to_wall, 0], [2 * dist_to_wall, 0]), 
            Polyhedron(np.array([[-2, -1], [2, -1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-2 * dist_to_wall, 0], [2 * dist_to_wall, 0]), 
            Polyhedron(np.array([[-2, -1], [2, -1]]), np.array([0, 0]))),
    box.Surface(Polyhedron.fromBounds([-dist_to_wall, 0], [-dist_to_wall, 2]),
            Polyhedron(np.array([[-1, -1], [-1, 1]]), np.array([0, 0]))),
]

env = box.Environment(surfaces, Polyhedron.fromBounds([-1.2, 0], [1.2, 2]))
opt = BoxAtlasContactStabilization(initial_state, env, num_time_steps=10,
                                  )
solnData = opt.solve()
ts = solnData.states.components[0].breaks

SolutionResult.kSolutionFound


In [165]:
box.planPlayback(treevis, solnData)

In [171]:
constraints = []
for j in range(len(ts) - 1):
    tspan = [ts[j], ts[j] + 0.5 * (ts[j + 1] - ts[j])]
    constraints.extend(posture_constraints(robot, solnData.states(ts[j]), tspan=tspan))
    
q_seed = xstar[:robot.number_of_positions()]
options = ik.IKoptions(robot)

q_costs = np.ones(robot.number_of_positions())
q_costs[:3] = 0
q_costs[4:5] = 10
q_costs[5] = 0
options.setQ(np.diag(q_costs))
results = ik.InverseKinPointwise(robot, ts[:-1], 
                                 np.repeat(q_seed, len(ts) - 1, axis=1), 
                                 np.repeat(q_seed, len(ts) - 1, axis=1), 
                                 constraints, options)
x_sol = [np.pad(q, (0, robot.number_of_velocities()), "constant") for q in results.q_sol]
pp = PiecewisePolynomial.FirstOrderHold([t * 5 for t in ts[:-1]], x_sol)

In [179]:
pp_0 = PiecewisePolynomial.ZeroOrderHold([0, 0.001], [x_sol[0], x_sol[0]])
drakevis.PlaybackTrajectory(pp_0)

In [180]:
drakevis.PlaybackTrajectory(pp)